In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os
import json
from file_handler import extract_text
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

def convertToJson() -> None:
  api_key = os.getenv("OPENROUTER_API_KEY")
  # api_base = os.getenv("OPENROUTER_API_BASE", "https://openrouter.ai/api/v1")
  # model = os.getenv("OPENROUTER_MODEL", "meta-llama/llama-3-8b-instruct")
  # Following steps to workout tomoorow
  #1. Use ipynb to upload a file and save it to a destination
  #2. Extract the content from pdf or docx
  # 3. Convert it to json using LLM
  # 4. Save the json to a file
  # 5. use the content of json to rewrite the cover letter 
  resume_text = extract_text("cv/my_cv.pdf")
  # client = OpenAI()



  schema = {
    "personal_information": {
      "name": "",
      "email": "",
      "phone": "",
      "location": ""
    },
    "summary": "",
    "skills": [],
    "experience": [
      {
        "role": "",
        "company": "",
        "start_date": "",
        "end_date": "",
        "description": []
      }
    ],
    "education": [
      {
        "degree": "",
        "institution": "",
        "year": ""
      }
    ]
  }

  # ⚙️ Initialize OpenRouter model (via OpenAI-compatible client)
  llm = ChatOpenAI(
      model="openai/gpt-4o-mini",   # can swap with other OpenRouter models
      api_key=os.getenv("OPENAI_API_KEY"),
      base_url="https://openrouter.ai/api/v1",
      temperature=0
  )

  # 📝 Prompt setup
  system = "You are a strict JSON extractor."
  human_prompt = """
  Extract the following CV into this JSON schema: {schema_content}

  Rules:
  - Copy content exactly into the schema.
  - Do not invent anything. Leave blank if missing.
  - Only return valid JSON.

  CV TEXT:
  {resume_text}
  """
  prompt = ChatPromptTemplate.from_messages([
      ("system", system),
      ("human", human_prompt)
  ])
  chain = prompt | llm

  # Run chain
  result = chain.invoke({"resume_text": resume_text,"schema_content": json.dumps(schema, indent=2)})

  # Parse model output
  try:
      if(result.content):
        extracted_json = json.loads(result.content)
        filename='cv/cv.json';
        with open(filename, "w") as file:
            json.dump(extracted_json, file, indent=2)
      else:
         print  ("❌ No content returned from model") 
      
  except json.JSONDecodeError:
      print("❌ Model returned invalid JSON:")
      print(result.content)

